In [36]:
:opt no-lint

# Tautology checker (주교재 8.6절)

In [52]:
type Nm = String -- 책에서는 Char이지만 우리는 String으로 하자

data Prop = Const Bool      -- True, False
          | Var Nm          -- x
          | Not Prop        -- ¬ A
          | And Prop Prop   -- A ∧ B
          | Imply Prop Prop -- A ⇒ B
          deriving Show

In [53]:
vA = Var "A"
vB = Var "B"
vC = Var "C"

p1 = And vA (Not vB) -- -- A /\ ㄱB
p2 = Imply vA vB -- A => B
p3 = Imply (And vA vB) vA -- (A /\ B) => A
p4 = Imply (And vA (Imply vA vB)) vB -- (A /\ (A=>B)) => B
p5 = Imply (And (Imply vA vB) (Imply vB vC)) (Imply vA vC)
     -- ((A=>B)/\(B=>C)) => (A=>C)

In [54]:
type Subst = [ (Nm, Bool) ] -- 덧셈식이나 람다식에서 Env에 해당

eval :: Subst -> Prop -> Bool
eval _ (Const b)   = b
eval s (Var x)     = find x s
eval s (Not p)     = not (eval s p)
eval s (And p q)   = eval s p && eval s q
eval s (Imply p q) = not(eval s p) || eval s q
          -- 책에서는 eval s p <= eval s q

find x s = case lookup x s of
             Nothing -> error $ x++" not defined"
             Just b -> b

In [55]:
import Data.List -- union 등

-- 책에서는 중복을 나중에 제거하지만 우리는 그냥 여기서 union을 쓰자
vars :: Prop -> [Nm]
vars (Const _)   = []
vars (Var x)     = [x]
vars (Not p)     = vars p
vars (And p q)   = vars p `union` vars q
vars (Imply p q) = vars p `union` vars q

In [56]:
vars p4
vars p5

["A","B"]

["A","B","C"]

In [57]:
bools :: Int -> [[Bool]]
bools n | n <= 0    = [[]]
        | otherwise = map (False:) bss ++ map (True:) bss
        -- | otherwise = [b:bs | b<-[False,True], bs<-bss]
        where bss = bools (n-1)

In [58]:
bools 2
putStrLn "-----"
mapM_ print $ bools 2

[[False,False],[False,True],[True,False],[True,True]]

-----

[False,False]
[False,True]
[True,False]
[True,True]

In [59]:
bools 3
putStrLn "-----"
mapM_ print $ bools 3

[[False,False,False],[False,False,True],[False,True,False],[False,True,True],[True,False,False],[True,False,True],[True,True,False],[True,True,True]]

-----

[False,False,False]
[False,False,True]
[False,True,False]
[False,True,True]
[True,False,False]
[True,False,True]
[True,True,False]
[True,True,True]

In [60]:
substs :: Prop -> [Subst]
substs p = map (zip vs) (bools (length vs))
        -- [zip vs bs | bs<-bools (length vs)]
        where vs = vars p

In [61]:
substs p4
putStrLn "-----"
mapM_ print $ substs p4

[[("A",False),("B",False)],[("A",False),("B",True)],[("A",True),("B",False)],[("A",True),("B",True)]]

-----

[("A",False),("B",False)]
[("A",False),("B",True)]
[("A",True),("B",False)]
[("A",True),("B",True)]

In [62]:
substs p5
putStrLn "-----"
mapM_ print $ substs p5

[[("A",False),("B",False),("C",False)],[("A",False),("B",False),("C",True)],[("A",False),("B",True),("C",False)],[("A",False),("B",True),("C",True)],[("A",True),("B",False),("C",False)],[("A",True),("B",False),("C",True)],[("A",True),("B",True),("C",False)],[("A",True),("B",True),("C",True)]]

-----

[("A",False),("B",False),("C",False)]
[("A",False),("B",False),("C",True)]
[("A",False),("B",True),("C",False)]
[("A",False),("B",True),("C",True)]
[("A",True),("B",False),("C",False)]
[("A",True),("B",False),("C",True)]
[("A",True),("B",True),("C",False)]
[("A",True),("B",True),("C",True)]

In [63]:
isTaut :: Prop -> Bool
isTaut p = and [eval s p | s <- substs p]

In [64]:
print p1
isTaut p1
print p2
isTaut p2
print p3
isTaut p3
print p4
isTaut p4
print p5
isTaut p5

And (Var "A") (Not (Var "B"))

False

Imply (Var "A") (Var "B")

False

Imply (And (Var "A") (Var "B")) (Var "A")

True

Imply (And (Var "A") (Imply (Var "A") (Var "B"))) (Var "B")

True

Imply (And (Imply (Var "A") (Var "B")) (Imply (Var "B") (Var "C"))) (Imply (Var "A") (Var "C"))

True